In [1]:
import numpy as np

In [228]:
X = np.array([[1,0,1,1,0],
             [0,1,1,1,0],
             [0,0,1,0,1]])

In [91]:
from sklearn.decomposition import NMF
model = NMF(n_components=3, init='random', random_state=0)

In [97]:
W = model.fit_transform(X)
H = model.components_

In [241]:
np.matrix.round(np.dot(W,H),3)

array([[1., 0., 1., 1., 0.],
       [0., 1., 1., 1., 0.],
       [0., 0., 1., 0., 1.]])

In [99]:
print(f"W: {np.shape(W)}")
print(f"H: {np.shape(H)}")

W: (3, 3)
H: (3, 5)


In [100]:
np.matrix.round(W,2)

array([[0.  , 0.25, 0.  ],
       [0.  , 0.  , 2.32],
       [0.63, 0.  , 0.  ]])

In [101]:
np.matrix.round(H,2)

array([[0.  , 0.  , 1.59, 0.  , 1.59],
       [3.96, 0.  , 3.96, 3.96, 0.  ],
       [0.  , 0.43, 0.43, 0.43, 0.  ]])

In [105]:
A = np.random.rand(3,3)
S = np.random.rand(3,5)

In [152]:
def normalizar(matriz):
    n, m = np.shape(matriz)
    for i in range(m):
        A[:,i] = A[:,i]/np.linalg.norm(A[:,i])
    return matriz
            
def norma(columna):
    retorno = 0
    for cosa in columna:
        retorno += cosa**2
    return np.sqrt(retorno)

def no_negativos(matriz):
    n, m = np.shape(matriz)
    for i in range(n):
        for j in range(m):
            if A[i][j] < 0:
                A[i][j] = 0
    return matriz

In [139]:
A

array([[0.1220032 , 0.35440706, 0.32720698],
       [0.55740239, 0.42532828, 0.92469341],
       [0.37215667, 0.1084151 , 0.46871878]])

In [156]:
maximo = 5
mu = 0.1
constante = 0

In [157]:
0.1220032**2 + 0.55740239**2 + 0.37215667**2

0.46408279221344106

In [165]:
A_t = A
S_t = np.transpose(S)
for i in range(maximo):
    delta = A_t - np.dot(mu*(np.dot(A_t, S_t) - X), np.transpose(S_t))
    delta = no_negativos(delta)
    delta = normalizar(delta)
    A_t = delta
    # St+1 = St ((At+1)T X) ./ ((At+1)T (At+1)St + λ)
    numerador = np.dot(np.transpose(A_t), X)
    denominador = np.dot(np.transpose(A_t), np.dot(A_t, S_t)) + constante
    derecha = numerador / denominador
    S_t = np.dot(S_t, derecha)
    
    print(denominador)

ValueError: shapes (3,3) and (5,3) not aligned: 3 (dim 1) != 5 (dim 0)

In [164]:
A/A

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [166]:
# X = UV

X

array([[1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 0, 1, 0, 1]])

In [167]:
def generar_matrices(X, k):
    n,m = np.shape(X)
    U = np.random.rand(n,k)
    V = np.transpose(np.random.rand(k,m))
    return U,V

In [242]:
U,V = generar_matrices(X,3)


In [227]:
W = model.fit_transform(X)
H = model.components_
np.round(H)

array([[0., 0., 2., 0., 2.],
       [4., 0., 4., 4., 0.],
       [0., 0., 0., 0., 0.]])

In [280]:
def ponderadores_U(X,U,V):
    """
    Función que recibe la matriz primaria X de n x m,
    la matriz U de n x k y la matriz V de m x k
    
    Calcula el ponderador para la actualizacion de la matriz U
    
    Retorna la matriz que ponderará a la matriz U
    """
    numerador = np.dot(X,V)
    denominador = np.dot(U,np.dot(np.transpose(V),V))
    return numerador/denominador

def ponderadores_V(X,U,V):
    """
    Función que recibe la matriz primaria X de n x m,
    la matriz U de n x k y la matriz V de m x k
    
    Calcula el ponderador para la actualizacion de la matriz V
    
    Retorna la matriz que ponderará a la matriz V
    """
    numerador = np.dot(np.transpose(X),U)
    denominador = np.dot(V,np.dot(np.transpose(U),U))
    return numerador/denominador

def actualizar_matriz(matriz, ponderador):
    """
    Función que recibe la matriz primaria de n x m y su
    matriz por la que será ponderada de las mismas dimensiones
    
    Pondera cada entrada (i,j) de la matriz por la respectiva
    casilla de la matriz ponderadora
    
    Retorna la matriz ya ponderada
    """
    a,b = np.shape(matriz), np.shape(ponderador)
    if (a != b):
        print(f"Error de dimensiones entre matriz: {a} y ponderador:{b}")
    n,m = a
    for i in range(n):
        for j in range(m):
            matriz[i,j] = matriz[i,j]*ponderador[i,j]
    return matriz

def normalizar_matrices(U,V):
    """
    Función que recibe la matriz U de n x k
    y la matriz V de m x k
    
    Normaliza las columnas de ambas matrices según la norma
    de la matriz U
    
    Retorna las matrices actualizadas
    """
    n, m = np.shape(U)
    for i in range(m):
        norma = np.linalg.norm(U[:,i])
        #V[:,i] = V[:,i]*norma
        U[:,i] = U[:,i]/norma
    return U,V

In [299]:
U,V = generar_matrices(X,4)

In [308]:
def nueva_funcion(X,U,V):
    U_t, V_t = U, V
    for i in range(100000):
        ponderador_U = ponderadores_U(X,U_t,V_t)
        ponderador_V = ponderadores_V(X,U_t,V_t)
        U_t = actualizar_matriz(U_t, ponderador_U)
        V_t = actualizar_matriz(V_t, ponderador_V)
        U_t, V_t = normalizar_matrices(U_t,V_t)

        error = np.linalg.norm(np.dot(U_t, np.transpose(V_t)) - X)

        if error < 0.001:
            return("Fin para: " + str(i))
            break
    return("Excede")

In [311]:
for _ in range(10):
    U,V = generar_matrices(X,4)
    print(nueva_funcion(X,U,V))

Fin para: 2955
Fin para: 8862
Fin para: 2510
Fin para: 6586
Fin para: 2697
Fin para: 3355
Fin para: 2203
Fin para: 3431
Fin para: 2155
Fin para: 4547


In [313]:
for _ in range(10):
    U,V = generar_matrices(X,3)
    print(nueva_funcion(X,U,V))

Fin para: 3072
Fin para: 3234
Fin para: 3244
Fin para: 3239
Fin para: 3217
Fin para: 2805
Fin para: 3226
Fin para: 3229
Fin para: 3198
Fin para: 3243


In [314]:
for _ in range(10):
    U,V = generar_matrices(X,5)
    print(nueva_funcion(X,U,V))

Fin para: 9235
Fin para: 6055
Fin para: 2561
Fin para: 7353
Fin para: 7568
Fin para: 5400
Fin para: 8411
Fin para: 17942
Fin para: 6604
Fin para: 6647


In [310]:
np.round(np.dot(U_t, np.transpose(V_t)),3)

array([[1.   , 0.001, 1.   , 1.   , 0.   ],
       [0.   , 1.   , 1.   , 1.   , 0.   ],
       [0.   , 0.   , 1.   , 0.001, 1.   ]])

In [319]:
for _ in range(10):
    U,V = generar_matrices(X,4)
    
    print(nueva_funcion(X,U*0.2,V*0.2))

Fin para: 4665
Fin para: 3873
Fin para: 2500
Fin para: 6411
Fin para: 8938
Fin para: 5104
Fin para: 3573
Fin para: 3668
Fin para: 2662
Fin para: 5729
